In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

import sys
from tifffile import imread
import h5py
import numpy as np

sys.path.append('/home/miguel/Projects/uni/phd/smlm_z/smlm-z/src/smlm_z/pipelines')
try:
    from preprocessing.nodes import extract_training_stacks
except ImportError:
    from preprocessing.nodes import extract_training_stacks

bead_stacks = [
    '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_5/3um_stack_20nm_step_20nm_bead_5_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_3/3um_stack_20nm_step_20nm_bead_3_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_9/3um_stack_20nm_step_20nm_bead_9_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_7/3um_stack_20nm_step_20nm_bead_7_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_6/3um_stack_20nm_step_20nm_bead_6_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_2/3um_stack_20nm_step_20nm_bead_2_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_4/3um_stack_20nm_step_20nm_bead_4_MMStack_Default.ome.tif',
#     '/home/miguel/Projects/uni/data/smlm_3d/20230331_beads/3um_stack_20nm_step_20nm_bead_8/3um_stack_20nm_step_20nm_bead_8_MMStack_Default.ome.tif',
]

all_spots = []
all_locs = []
all_stacks = []
for bead_stack_path in bead_stacks:
    bead_stack = imread(bead_stack_path)
    slice_path = bead_stack_path.replace('.ome', '_slice.ome')
    spots_path = slice_path.replace('.tif', '_spots.hdf5')
    with h5py.File(spots_path) as f:
        spots = np.array(f['spots'])

    locs_path = spots_path.replace('_spots', '_locs')
    locs = pd.read_hdf(locs_path, key='locs')

    from skimage.feature import match_template

    parameters = {
        'picasso': {
            'spot_size': 31,
            'localised_frame': bead_stack.shape[0]//2
        },
        'DEBUG': False
    }
    stacks = extract_training_stacks(spots, bead_stack, parameters)

    all_spots.append(spots)
    all_locs.append(locs)
    all_stacks.append(stacks)

min_stack_length = min(list(map(lambda s: s.shape[1], all_stacks)))
all_stacks = [s[:, :min_stack_length] for s in all_stacks]

spots = np.concatenate(all_spots)
locs = pd.concat(all_locs)
stacks = np.concatenate(all_stacks)

print(spots.shape)
print(locs.shape)
print(stacks.shape)
max_pixel_val = np.max(stacks, axis=(1,2,3))
mean_pixel_val = np.median(stacks, axis=(1,2,3))
snr = max_pixel_val / mean_pixel_val
locs['snr'] = snr

locs = locs.reset_index(drop=True)

# for i in np.argsort(snr):
#     if snr[i] > 3:
#         show_psf_axial(stacks[i], snr[i])
    
plt.rcParams["figure.figsize"] = (4, 4)

import seaborn as sns
sns.scatterplot(data=locs[locs['snr'] > 3], x='x', y='y', hue='snr')
plt.xlim(locs['x'].min(), locs['x'].max())
plt.ylim(locs['y'].min(), locs['y'].max())
plt.show()


idx = locs['snr'] > 3
locs = locs[idx]
spots = spots[idx]
stacks = stacks[idx, :150]
stacks = np.stack([mask_psf(s, 1, 12) for s in stacks])


In [ ]:
try:
    from smlm_z.pipelines.preprocessing.nodes import resize_stacks, norm_stacks
except ImportError:
    from smlm_z.pipelines.preprocessing.nodes import resize_stacks, norm_stacks
stacks_min, stacks_max, stacks_dtype = stacks.min(), stacks.max(), stacks.dtype
# stacks = resize_stacks(stacks, target_shape=(stacks.shape[1], 32, 32))
stacks = norm_stacks(stacks)
print(stacks.min(), stacks.max())
# stacks -= stacks.min()
# stacks = stacks / stacks.max()
# stacks *= (stacks_max - stacks_min)
# stacks += stacks_min
# stacks = stacks.astype(stacks_dtype)


In [ ]:
for i in range(10):
    target_psf = stacks[i]
    show_psf_axial(target_psf, str(i))

In [ ]:
from pyotf.utils import prep_data_for_PR
target_spot = 5

target_psf = stacks[target_spot].copy()
df = locs

z_step = 0.02
z_range = target_psf.shape[0] * z_step / 2
print(target_psf.shape)
print(z_step)
print(z_range)

optical_params = dict(Nn=target_psf.shape[1], 
                          pixel_size=0.110, 
                          zrange=z_range, 
                          dz=z_step, 
                          magnification=111.11,
                          ill_NA=1.4,
                          det_NA=1.3,
                          n=1.335,
#                           fwhmz=3.0,
                          ill_wavelength=635,
                          det_wavelength=635)



print(target_psf.min(), target_psf.max())
show_psf_axial(target_psf)


# fit_psf(target_psf, optical_params, n_coefs=16, lr=1e-5)

In [ ]:
print(np.mean(errors))

In [ ]:
df = pd.DataFrame.from_records(results)
print(df)
import seaborn as sns
sns.boxplot(data=df, x='opt', y='error')
plt.show()

sns.boxplot(data=df, x='opt', y='error')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

plt.rcParams["figure.figsize"] = (10, 3)

df = pd.DataFrame.from_records(results)
print(f'{df["error"].mean():.4E}')
df['param_sum'] = df['params'].apply(lambda x: sum(abs(np.array(x[1]))))

print(df.shape)
df = df[df['param_sum'] < 100]
print(df.shape)
idx = df['error'] <= 1
df = df[idx]


corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

sns.scatterplot(data=df, x='param_sum', y='error')
plt.yscale('log')
plt.show()



for c in list(df):
    if 'coef' in c:
        f, axs = plt.subplots(1, 2)
        sns.scatterplot(data=df, x='x', y='y', hue=c, ax=axs[0])
        sns.scatterplot(data=df, x='x', y='y', hue='error', ax=axs[1])
        plt.show()

In [ ]:
df2 = df.copy()
df2['x'] -= 450
df2['y'] -= 450

# Define a function to convert cartesian coordinates to polar coordinates
def cartesian_to_polar(x, y):
    radius = np.sqrt(x**2 + y**2)
    angle = np.arctan2(y, x)
    return radius, angle

# Apply the conversion function to each row of the DataFrame
df2['r'], df2['phi'] = zip(*df2.apply(lambda row: cartesian_to_polar(row['x'], row['y']), axis=1))

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('gr',['g', 'black', 'r'], N=64) 

plt.rcParams["figure.figsize"] = (10, 3)

for c in list(df):
    if 'coef' in c or 'offset' in c:
        f, axs = plt.subplots(1, 2)
        if 'coef' in c:
            coef_id = int(c.split('_')[-1]) + 1
            title = poppy.zernike.zern_name(coef_id)
        else:
            title = 'offset'
        plt.title(title)
        sns.scatterplot(data=df2, x='phi', y=c, hue='error', ax=axs[0], palette=cmap)
        sns.scatterplot(data=df2, x='r', y=c, hue='error', ax=axs[1], palette=cmap)
        plt.show()
        

In [ ]:
df['x_norm'] = norm_zero_one(df['x'])
df['y_norm'] = norm_zero_one(df['y'])

In [ ]:
show_psf_axial(stacks[4])

In [ ]:
from smlm_z.pipelines.preprocessing.nodes import norm_images, resize_stacks
from data.estimate_offset import get_peak_sharpness

# Eval data
stack_psfs = stacks

val_zs = []
val_xy = []
val_psfs = []
for i in range(stack_psfs.shape[0]):
    psf = stack_psfs[i]
    
#     peak = int(get_peak_sharpness(psf))
#     z_range = 1.0
#     n_slices_from_focus = int(z_range / z_step)
#     start = max((peak-n_slices_from_focus, 0))
#     end = min((peak+n_slices_from_focus, psf.shape[0]))
    
#     psf = psf[start:end]
    peak = 0
    val_zs.append((np.arange(psf.shape[0])-peak) * z_step * 1000)
    xy_coords = df.iloc[i][['x_norm', 'y_norm']].tolist()
#     xy_coords = [0, 0]
    val_xy.append(xy_coords)
    val_psfs.append(psf.astype(float))
#     show_psf_axial(stack_psfs[i])

val_xy = np.array(val_xy)
val_zs = np.array(val_zs)
for i in range(len(val_psfs)):
    val_psfs[i] = norm_images(val_psfs[i])
    val_psfs[i] = val_psfs[i][:, :, :, np.newaxis]
    val_psfs[i] = np.repeat(val_psfs[i], 3, axis=-1)

# show_psf_axial(val_psfs[i].mean(axis=-1))

print(val_xy.shape)
print(val_zs.shape)

In [ ]:
from tqdm import tqdm
train_psfs = []
train_zs = []
train_xy = []
for entry in df.to_dict(orient='records'):
    psf = simul.get_scalar_psf(offset=0, zern_coefs=entry['params'][1])
    plt.rcParams["figure.figsize"] = (10, 10)
    show_psf_axial(psf)

    psf = norm_images(psf)[:, :, :, np.newaxis]
    psf = np.repeat(psf, 3, axis=-1)
    zs = np.linspace(-simul.zrange, simul.zrange-simul.dz, psf.shape[0]) * 1000
    train_psfs.append(psf)
    train_zs.append(zs)
    xy_coords = [entry['x_norm'], entry['y_norm']]
    for _ in range(psf.shape[0]):
        train_xy.append(xy_coords)

train_psfs = np.concatenate(train_psfs)

train_xy = np.array(train_xy)
train_zs = np.array(train_zs).flatten()[:, np.newaxis]
print(train_psfs.shape)
print(train_xy.shape)
print(train_zs.shape)

X = (train_psfs, train_xy)
y = train_zs

In [ ]:
params = {
    'train_size': 0.8, 
    'test_size': 0.1, 
    'val_size': 0.1, 
    'random_seed': 42,
    'training': {
        'learning_rate': 1e-4,
        'batch_size': 128,
        'max_epochs': 1000,
        'aug_ratio': 4
    },
    'data_augmentation': {
        'noise_std': 0.3,
        'max_lateral_translation_px': 2,
    },
    'model_input_shape': [32, 32, 3],
    
}

In [ ]:
%load_ext autoreload
%autoreload 2
from smlm_z.pipelines.train_model.nodes import split_train_val_test, train_classifier, check_data, augment_datasets, check_data

X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test(X, y, params)

print(X_train[0].shape, X_train[1].shape, y_train.shape)
X_train, y_train = augment_datasets(X_train, y_train, params)
print(X_train[0].shape, X_train[1].shape, y_train.shape)
plt.rcParams["figure.figsize"] = (3, 3)
# check_data(X_train, y_train, X_val, y_val, X_test, y_test)

X_exp, y_exp = (np.concatenate(val_psfs), val_xy), np.concatenate(val_zs)
print(X_exp[0].shape, X_exp[1].shape, y_exp.shape)

check_data(X_train, y_train, X_val, y_val, X_exp, y_exp)

In [ ]:
import os
import pickle
            
LOAD=True

if LOAD:
    for fpath in os.listdir('./backup'):
        varname = fpath.split('.')[0]
        if 'pickle' in fpath:
            with open(f'./backup/{fpath}', 'rb') as f:
                globals()[varname] = pickle.load(f)
        
            
print(X_train[0].shape, X_train[1].shape)
print(y_train.shape, y_train.shape)


%load_ext autoreload
%autoreload 2
from smlm_z.pipelines.train_model.nodes import split_train_val_test, train_classifier, check_data, augment_datasets, check_data

X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test(X, y, params)


X_exp, y_exp = (np.concatenate(val_psfs), val_xy), np.concatenate(val_zs)
print(X_exp[0].shape, X_exp[1].shape, y_exp.shape)

check_data(X_train, y_train, X_val, y_val, X_exp, y_exp)


In [ ]:
%load_ext autoreload
%autoreload 2

import keras
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from pyotf.utils import prep_data_for_PR
from smlm_z.pipelines.train_model.nodes import train_classifier
import numpy as np
from tqdm import tqdm

plt.rcParams["figure.figsize"] = (5, 5)

def eval_bead_stack(model):

    errors = []
    i = 0 
    for stack, xy, zs in zip(val_psfs, val_xy, val_zs):
        xy = np.repeat(xy[np.newaxis, :], stack.shape[0], 0)
        stack -= np.median(stack)
    #     stack2 = (stack * 255 * 255).mean(axis=-1).astype(int)
    #     stack2 = prep_data_for_PR(stack2, multiplier=1.01)
    #     stack2 = norm_images(stack2)
    #     stack2 = np.repeat(stack2[:, :, :, np.newaxis], 3, axis=-1)
        pred = model.predict((stack, xy), verbose=0)
        pred -= pred.min()
        error = mean_absolute_error(pred, zs)
        errors.append(float(error))
        plt.plot(zs, pred, alpha=0.2)
    plt.show()

    errors = np.mean(errors)
    tqdm.write(f'MAE: {errors:.3E}')
    
        

class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 25 == 0:
            eval_bead_stack(self.model)

custom_callback = myCallback()



model, _ = train_classifier(X_train, y_train, X_val, y_val, params, extra_callbacks=[custom_callback])

model_file = './backup/model'
model.save(model_file)

In [ ]:
from data.visualise import grid_psfs, show_psf_axial

for i in range(10):
    show_psf_axial(val_psfs[i].mean(axis=-1))

In [ ]:


from tensorflow import keras
model = keras.models.load_model(model_file)



    

In [ ]:
from smlm_z.pipelines.train_model.nodes import eval_classifier
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3, 3)
# Eval on training data
eval_classifier(model, X_train, y_train, X_val, y_val, X_test, y_test)

In [ ]:

def eval_bead_stack(model):

    errors = []
    i = 0 
    for stack, xy, zs in zip(val_psfs, val_xy, val_zs):
        xy = np.repeat(xy[np.newaxis, :], stack.shape[0], 0)
        stack -= np.median(stack)
#         stack = (stack * 255 * 255).mean(axis=-1).astype(int)
#         stack = prep_data_for_PR(stack, multiplier=1.01)
#         stack2 = norm_images(stack2)
    #     stack2 = np.repeat(stack2[:, :, :, np.newaxis], 3, axis=-1)
        pred = model.predict((stack, xy), verbose=0)
        zs -= zs.min()
        pred -= pred.min()
        error = mean_absolute_error(pred, zs)
        errors.append(float(error))
        plt.plot(zs, pred)
        plt.show()
        show_psf_axial(stack.mean(axis=-1), '', 1)
    errors = np.mean(errors)
    tqdm.write(f'MAE: {errors:.3E}')
eval_bead_stack(model)